<a href="https://colab.research.google.com/github/SijieQiu/RQ/blob/main/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Converted into json file**

In [1]:
from google.colab import files

uploaded = files.upload()

Saving 127.docx to 127.docx


In [2]:
import json
!pip install python-docx
from docx import Document

file_name = '127.docx'

doc = Document(file_name)

data = []

for para in doc.paragraphs:
    if para.text.strip():
        parts = para.text.split('\t')
        if len(parts) >= 2:
            content = parts[0].strip()
            labels = [label.strip() for label in parts[1].split(',')]
            data.append({
                "content": content,
                "labels": labels
            })
print(f"Number of documents: {len(data)}")
print(f"Sample document: {data[0]}")

json_file_name = '127.json'
with open(json_file_name, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

print(f"JSON file '{json_file_name}' created successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.2 MB/s eta 0:00:00
Number of documents: 27
Sample document: {'content': 'Joe Potter and Matthew Trent work at a small, remote outpost in the middle of a snowy wilderness. Joe wakes up on Christmas Day to find Matt preparing Christmas dinner, with "I Wish It Could Be Christmas Everyday" playing on the radio. Matt tries to get Joe to talk about why he accepted the job at the outpost, a topic they have never discussed in the five years they have worked together. Joe is reluctant to say anything and instead asks why Matt took the job. Happy with the conversation, Matt begins his story. The story is set in a world where people can access the Internet through an augmented reality device implanted in their eyes called the "Z-Eye". Matt was once a dating coach who taught seduction techniques to single men who struggle to attract women. He directs one of his clients, the shy and socially awkward Harry, into gatecrashing an office Ch

**Labelled LDA**

In [6]:
!pip install tomotopy
import tomotopy as tp
import json
from collections import Counter

file_name = '127.json'

# Load the data from the specified file
with open(file_name, 'r') as input_file:
    data = json.load(input_file)

# Collect unique labels from the dataset
unique_labels = set()
for index, row in enumerate(data):
    labels = set(row['labels'])
    unique_labels |= set(labels)

unique_label_count = len(unique_labels)
# Use 'latent_topics' instead of 'k'
mdl = tp.PLDAModel(latent_topics=unique_label_count, min_cf=10, rm_top=5)

# Add documents to the model
for index, row in enumerate(data):
    if index % 1000 == 0 and index > 0:
        print('Adding document #%s' % index)
    content = row['content']
    labels = row['labels']
    clean_document = [token for token in content.strip().split() if len(token) > 2]
    if clean_document:  # Ensure the document is not empty
        mdl.add_doc(clean_document, labels=labels)

print('Starting to train the model')
for i in range(0, 100, 10):
    mdl.train(10)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))

# Print top words for each topic
for k in range(mdl.latent_topics): # Use mdl.latent_topics instead of mdl.k
    if k in mdl.topic_label_dict:
        print('Top 10 words of global topic #{} {}: {}'.format(k, mdl.topic_label_dict[k], mdl.get_topic_words(k, top_n=10)))
    else:
        print('Top 10 words of global topic #{}: {}'.format(k, mdl.get_topic_words(k, top_n=10)))

Starting to train the model
Iteration: 0	Log-likelihood: -7.195746620912574
Iteration: 10	Log-likelihood: -6.986593891078235
Iteration: 20	Log-likelihood: -6.8762905875902725
Iteration: 30	Log-likelihood: -6.751808439356662
Iteration: 40	Log-likelihood: -6.716789590348613
Iteration: 50	Log-likelihood: -6.6403457994998805
Iteration: 60	Log-likelihood: -6.533191028624346
Iteration: 70	Log-likelihood: -6.486579509281423
Iteration: 80	Log-likelihood: -6.443705556253121
Iteration: 90	Log-likelihood: -6.398676324295042
Top 10 words of global topic #0: [('Clayton', 0.3894059956073761), ('take', 0.11863259971141815), ('able', 0.11863259971141815), ('where', 0.06786258518695831), ('how', 0.06786258518695831), ('body', 0.03401590883731842), ('pulls', 0.03401590883731842), ('begs', 0.017092570662498474), ('too', 0.017092570662498474), ('get', 0.017092570662498474)]
Top 10 words of global topic #1: [('Daly', 0.443613201379776), ('Cole', 0.3327791690826416), ('game', 0.0556940957903862), ('promises

# **Cleaning data**

In [12]:
# Install tomotopy if not already installed
!pip install tomotopy

import tomotopy as tp
import json
from collections import Counter

# Define the stop words
stop_words = set([
    'the', 'to', 'a', 'and', 'is', 'of', 'in', 'that', 'his', 'with',
    'her', 'she', 'he', 'it', 'on', 'they', 'this', 'for', 'but', 'from',
    'by', 'as', 'at', 'if', 'not', 'or', 'their', 'be', 'out', 'about',
    'an', 'while', 'comes', 'starts', 'him', 'him.', 'Ashley', 'David', 'Cliff',
    'Lana', 'Rolo', 'Carrie', 'Jack', 'Rachel', 'Catherine', 'Nida', 'Pia', 'Victoria',
    'says', 'asks', 'has', 'tells', 'was', 'get', 'gets', 'her.', "doesn't", 'into', 'who',
    'what', "it's", 'have', 'can', 'Emily', 'Danny', 'Karl', 'Martha', 'Ash',
    'Jem', 'Bing', 'Mazey', 'Hector', 'Duke', 'Joe', 'Matt', 'Beth', 'Parke',
    'Baxter', 'Jamie', 'Waldo', 'Liam', 'Sara', 'Mia', 'Shazia', 'Rob', 'Amy',
    'Frank', 'Bella', 'Nish', 'Dawson', 'Clayton', 'Parker', 'Callow', 'Ffion',
    'Daly', 'Cole', 'Katie', 'Jessica', 'Henry', 'I', 'you', 'we', 'he', 'she',
    'it', 'they', 'me', 'him', 'her', 'us', 'them', 'my', 'your', 'our', 'his',
    'their', 'in', 'on', 'at', 'by', 'with', 'about', 'against', 'between',
    'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from',
    'up', 'down', 'off', 'over', 'under', 'have', 'has', 'had', 'do', 'does',
    'did', 'can', 'could', 'would', 'should', 'will', 'shall', 'might', 'must',
    'it.', 'them', 'are', 'like', 'one', 'where', 'then', 'just', 'because',
    'wants', 'watches', 'back', 'did', 'which', 'first', 'following', 'seen',
    'sees', 'when', 'When', 'how', 'see', 'take', 'she\'s', 'tries', 'all', 'more',
    'he\'s', 'looks', 'goes', 'been', 'put', 'find', 'didn\'t', 'they\'re', 'that\'s', 'herself',
    'Bing', 'without', 'three', 'day', 'using', 'several', 'every', 'look',
    'David\'s', 'he\'ll', 'want', 'know', 'hit', 'left', 'again', 'telling', 'makes', 'realizes',
    'trying', 'part', 'talking',  'took', 'Jamie', 'Waldo', 'coming', 'suggests', 'himself',
    'later', 'having', 'out', 'run', 'its', 'Jessica', 'what\'s',
    'Danny', 'Karl',  'asking', 'tried', 'day', 'sits', 'say', 'says', 'asks',
    'She\'s', 'Sandy', 'Mona', 'Mac', 'show', 'glass', 'until', 'only', 'live', 'called', 'few', 'used', 'Liam',
    'Ffion',  'around', 'meets', 'already', 'eventually', 'explains', 'takes', 'help', 'behind', 'allows', 'plays', 'opens', 'allowing',
    'Nida', 'Gaap', 'Vicky', 'Len', 'Michael', 'Keith', 'Suzie', 'Chris', 'three', 'Joan', 'Salma', 'Krish', 'Eric',
    'She\'s', 'Sandy', 'Mona', 'Mac', 'show', 'watch', 'show.', 'Jack', 'Rachel', 'Catherine', 'Too', 'Munk', 'Kevin', 'music', 'new', 'still', 'play', 'Mia',
    'Shazia', 'Rob', 'picture', 'puts', 'promises', 'woman', 'case', 'grabs', 'phone', 'decides', 'try', 'reveals', 'new', 'Victoria', 'Baxter',
    'nearby', 'calls', 'Bear', 'Jamie', 'public', 'own', 'world', 'Liam', 'back.', 'Kelly', 'time', 'leaves.', 'party', 'real', 'her,'
    'agrees', 'working',  'again.', 'Parke', 'becomes', 'Jem', 'another', 'house',' onto', 'Jamie', 'Ffion', 'Duke', 'Cole','Bing','Katie','herself',
    'together.', 'Duke', 'Cole', 'Katie', 'really', 'said', 'other', 'leave', 'months', 'monkey',
    'Carrie', 'Emily', 'feel', 'Dawson', 'day', 'follows', 'that', 'hasn\'t', 'Daly', 'Baxter',
    'Chris', 'Waldo', 'Jem', 'Liam', 'Rachel', 'Martha', 'Amy', 'Too', 'Kappa', 'Nish', 'Bella',
    'Rolo', 'Pia', 'Gaap', 'Keith', 'Karl', 'Vicky', 'Catherine', 'Coach', 'Salma', 'Kevin',
    'Victoria', 'Yorkie', 'San', 'work', 'killed', 'day', 'that', 'herself', 'suddenly', 'uses'
])

file_name = '127.json'

# Load the data from the specified file
with open(file_name, 'r') as input_file:
    data = json.load(input_file)

# Collect unique labels from the dataset
unique_labels = set()
for index, row in enumerate(data):
    labels = set(row['labels'])
    unique_labels |= set(labels)

unique_label_count = len(unique_labels)
# Use 'latent_topics' instead of 'k'
mdl = tp.PLDAModel(latent_topics=unique_label_count, min_cf=10, rm_top=5)

# Preprocess document content and add documents to the model
def preprocess_document(text):
    tokens = text.strip().split()
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words and len(word) > 2]
    return filtered_tokens

for index, row in enumerate(data):
    if index % 1000 == 0 and index > 0:
        print('Adding document #%s' % index)
    content = row['content']
    labels = row['labels']
    clean_document = preprocess_document(content)
    if clean_document:  # Ensure the document is not empty
        mdl.add_doc(clean_document, labels=labels)

print('Starting to train the model')
for i in range(0, 100, 10):
    mdl.train(10)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))

# Print top words for each topic
for k in range(mdl.latent_topics): # Use mdl.latent_topics instead of mdl.k
    if k in mdl.topic_label_dict:
        print('Top 10 words of global topic #{} {}: {}'.format(k, mdl.topic_label_dict[k], mdl.get_topic_words(k, top_n=10)))
    else:
        print('Top 10 words of global topic #{}: {}'.format(k, mdl.get_topic_words(k, top_n=10)))


Starting to train the model
Iteration: 0	Log-likelihood: -6.540682159321322
Iteration: 10	Log-likelihood: -6.3786934698840705
Iteration: 20	Log-likelihood: -6.256943118025001
Iteration: 30	Log-likelihood: -6.18793922593139
Iteration: 40	Log-likelihood: -6.097707359166726
Iteration: 50	Log-likelihood: -5.947104126858505
Iteration: 60	Log-likelihood: -6.0092225574142875
Iteration: 70	Log-likelihood: -5.958716485556394
Iteration: 80	Log-likelihood: -5.8660526884408695
Iteration: 90	Log-likelihood: -5.8587891069966025
Top 10 words of global topic #0: [('herself.', 0.0032051282469183207), ('dead', 0.0032051282469183207), ('onto', 0.0032051282469183207), ('Duke', 0.0032051282469183207), ('Cole', 0.0032051282469183207), ('Jamie', 0.0032051282469183207), ('Ffion', 0.0032051282469183207), ('Bing', 0.0032051282469183207), ('Katie', 0.0032051282469183207), ('memory', 0.0032051282469183207)]
Top 10 words of global topic #1: [('herself.', 0.0032051282469183207), ('dead', 0.0032051282469183207), ('o

**BERTopic**

In [13]:
import json
from sklearn.feature_extraction.text import CountVectorizer

# 读取JSON文件
file_name = '127.json'

with open(file_name, 'r') as input_file:
    data = json.load(input_file)

# 提取文档和标签
docs = [item['content'] for item in data]
labels = [item['labels'] for item in data]

# 打印文档和标签的数量，以及一个示例文档和它的标签
print(f"Number of documents: {len(docs)}")
print(f"Number of labels: {len(labels)}")
print(f"Sample document: {docs[0]}")
print(f"Sample label: {labels[0]}")

# 使用CountVectorizer将文本转换为向量
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(docs)

print(f"Feature shape: {X.shape}")

Number of documents: 27
Number of labels: 27
Sample document: Joe Potter and Matthew Trent work at a small, remote outpost in the middle of a snowy wilderness. Joe wakes up on Christmas Day to find Matt preparing Christmas dinner, with "I Wish It Could Be Christmas Everyday" playing on the radio. Matt tries to get Joe to talk about why he accepted the job at the outpost, a topic they have never discussed in the five years they have worked together. Joe is reluctant to say anything and instead asks why Matt took the job. Happy with the conversation, Matt begins his story. The story is set in a world where people can access the Internet through an augmented reality device implanted in their eyes called the "Z-Eye". Matt was once a dating coach who taught seduction techniques to single men who struggle to attract women. He directs one of his clients, the shy and socially awkward Harry, into gatecrashing an office Christmas party, and communicates with him through the Z-Eye. They are then 

In [14]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 M

In [20]:
import json
from bertopic import BERTopic
from bertopic.backend import BaseEmbedder
from bertopic.cluster import BaseCluster
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.feature_extraction.text import CountVectorizer # Import CountVectorizer

# Define the stop words
stop_words = set([
    'the', 'to', 'a', 'and', 'is', 'of', 'in', 'that', 'his', 'with',
    'her', 'she', 'he', 'it', 'on', 'they', 'this', 'for', 'but', 'from',
    'by', 'as', 'at', 'if', 'not', 'or', 'their', 'be', 'out', 'about',
    'an', 'while', 'comes', 'starts', 'him', 'him.', 'Ashley', 'David', 'Cliff',
    'Lana', 'Rolo', 'Carrie', 'Jack', 'Rachel', 'Catherine', 'Nida', 'Pia', 'Victoria',
    'says', 'asks', 'has', 'tells', 'was', 'get', 'gets', 'her.', "doesn't", 'into', 'who',
    'what', "it's", 'have', 'can', 'Emily', 'Danny', 'Karl', 'Martha', 'Ash',
    'Jem', 'Bing', 'Mazey', 'Hector', 'Duke', 'Joe', 'Matt', 'Beth', 'Parke',
    'Baxter', 'Jamie', 'Waldo', 'Liam', 'Sara', 'Mia', 'Shazia', 'Rob', 'Amy',
    'Frank', 'Bella', 'Nish', 'Dawson', 'Clayton', 'Parker', 'Callow', 'Ffion',
    'Daly', 'Cole', 'Katie', 'Jessica', 'Henry', 'I', 'you', 'we', 'he', 'she',
    'it', 'they', 'me', 'him', 'her', 'us', 'them', 'my', 'your', 'our', 'his',
    'their', 'in', 'on', 'at', 'by', 'with', 'about', 'against', 'between',
    'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from',
    'up', 'down', 'off', 'over', 'under', 'have', 'has', 'had', 'do', 'does',
    'did', 'can', 'could', 'would', 'should', 'will', 'shall', 'might', 'must',
    'it.', 'them', 'are', 'like', 'one', 'where', 'then', 'just', 'because',
    'wants', 'watches', 'back', 'did', 'which', 'first', 'following', 'seen',
    'sees', 'when', 'When', 'how', 'see', 'take', 'she\'s', 'tries', 'all', 'more',
    'he\'s', 'looks', 'goes', 'been', 'put', 'find', 'didn\'t', 'they\'re', 'that\'s', 'herself',
    'Bing', 'without', 'three', 'day', 'using', 'several', 'every', 'look',
    'David\'s', 'he\'ll', 'want', 'know', 'hit', 'left', 'again', 'telling', 'makes', 'realizes',
    'trying', 'part', 'talking',  'took', 'Jamie', 'Waldo', 'coming', 'suggests', 'himself',
    'later', 'having', 'out', 'run', 'its', 'Jessica', 'what\'s',
    'Danny', 'Karl',  'asking', 'tried', 'day', 'sits', 'say', 'says', 'asks',
    'She\'s', 'Sandy', 'Mona', 'Mac', 'show', 'glass', 'until', 'only', 'live', 'called', 'few', 'used', 'Liam',
    'Ffion',  'around', 'meets', 'already', 'eventually', 'explains', 'takes', 'help', 'behind', 'allows', 'plays', 'opens', 'allowing',
    'Nida', 'Gaap', 'Vicky', 'Len', 'Michael', 'Keith', 'Suzie', 'Chris', 'three', 'Joan', 'Salma', 'Krish', 'Eric',
    'She\'s', 'Sandy', 'Mona', 'Mac', 'show', 'watch', 'show.', 'Jack', 'Rachel', 'Catherine', 'Too', 'Munk', 'Kevin', 'music', 'new', 'still', 'play', 'Mia',
    'Shazia', 'Rob', 'picture', 'puts', 'promises', 'woman', 'case', 'grabs', 'phone', 'decides', 'try', 'reveals', 'new', 'Victoria', 'Baxter',
    'nearby', 'calls', 'Bear', 'Jamie', 'public', 'own', 'world', 'Liam', 'back.', 'Kelly', 'time', 'leaves.', 'party', 'real', 'her,'
    'agrees', 'working',  'again.', 'Parke', 'becomes', 'Jem', 'another', 'house',' onto', 'Jamie', 'Ffion', 'Duke', 'Cole','Bing','Katie','herself',
    'together.', 'Duke', 'Cole', 'Katie', 'really', 'said', 'other', 'leave', 'months', 'monkey',
    'Carrie', 'Emily', 'feel', 'Dawson', 'day', 'follows', 'that', 'hasn\'t', 'Daly', 'Baxter',
    'Chris', 'Waldo', 'Jem', 'Liam', 'Rachel', 'Martha', 'Amy', 'Too', 'Kappa', 'Nish', 'Bella',
    'Rolo', 'Pia', 'Gaap', 'Keith', 'Karl', 'Vicky', 'Catherine', 'Coach', 'Salma', 'Kevin',
    'Victoria', 'Yorkie', 'San', 'work', 'killed', 'day', 'that', 'herself', 'suddenly', 'uses'
])

# Load the JSON file
with open('127.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract documents and labels
docs = [item['content'] for item in data]
y_labels = [item['labels'][0] for item in data]  # Assuming each doc has at least one label and taking the first label

# Create a mapping from labels to integers
label_mapping = {label: i for i, label in enumerate(set(y_labels))}

# Convert labels to integers
y = [label_mapping[label] for label in y_labels]

# Prepare our empty sub-models and reduce frequent words while we are at it.
empty_embedding_model = BaseEmbedder()
empty_dimensionality_model = BaseDimensionalityReduction()
empty_cluster_model = BaseCluster()
# Use CountVectorizer to handle stop words
vectorizer_model = CountVectorizer(stop_words=list(stop_words)) # Convert stop_words set to a list
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
# Fit BERTopic without actually performing any clustering
topic_model = BERTopic(
    embedding_model=empty_embedding_model,
    umap_model=empty_dimensionality_model,
    hdbscan_model=empty_cluster_model,
    ctfidf_model=ctfidf_model, # Pass the ctfidf model
    vectorizer_model=vectorizer_model # Pass the CountVectorizer instance to BERTopic
)
topics, probs = topic_model.fit_transform(docs, y=y)  # Pass the integer labels to fit_transform


In [21]:
import json
from bertopic import BERTopic
from bertopic.backend import BaseEmbedder
from bertopic.cluster import BaseCluster
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.feature_extraction.text import CountVectorizer  # Import CountVectorizer

# Define the stop words
stop_words = set([
    'the', 'to', 'a', 'and', 'is', 'of', 'in', 'that', 'his', 'with',
    'her', 'she', 'he', 'it', 'on', 'they', 'this', 'for', 'but', 'from',
    'by', 'as', 'at', 'if', 'not', 'or', 'their', 'be', 'out', 'about',
    'an', 'while', 'comes', 'starts', 'him', 'him.', 'Ashley', 'David', 'Cliff',
    'Lana', 'Rolo', 'Carrie', 'Jack', 'Rachel', 'Catherine', 'Nida', 'Pia', 'Victoria',
    'says', 'asks', 'has', 'tells', 'was', 'get', 'gets', 'her.', "doesn't", 'into', 'who',
    'what', "it's", 'have', 'can', 'Emily', 'Danny', 'Karl', 'Martha', 'Ash',
    'Jem', 'Bing', 'Mazey', 'Hector', 'Duke', 'Joe', 'Matt', 'Beth', 'Parke',
    'Baxter', 'Jamie', 'Waldo', 'Liam', 'Sara', 'Mia', 'Shazia', 'Rob', 'Amy',
    'Frank', 'Bella', 'Nish', 'Dawson', 'Clayton', 'Parker', 'Callow', 'Ffion',
    'Daly', 'Cole', 'Katie', 'Jessica', 'Henry', 'I', 'you', 'we', 'he', 'she',
    'it', 'they', 'me', 'him', 'her', 'us', 'them', 'my', 'your', 'our', 'his',
    'their', 'in', 'on', 'at', 'by', 'with', 'about', 'against', 'between',
    'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from',
    'up', 'down', 'off', 'over', 'under', 'have', 'has', 'had', 'do', 'does',
    'did', 'can', 'could', 'would', 'should', 'will', 'shall', 'might', 'must',
    'it.', 'them', 'are', 'like', 'one', 'where', 'then', 'just', 'because',
    'wants', 'watches', 'back', 'did', 'which', 'first', 'following', 'seen',
    'sees', 'when', 'When', 'how', 'see', 'take', 'she\'s', 'tries', 'all', 'more',
    'he\'s', 'looks', 'goes', 'been', 'put', 'find', 'didn\'t', 'they\'re', 'that\'s', 'herself',
    'Bing', 'without', 'three', 'day', 'using', 'several', 'every', 'look',
    'David\'s', 'he\'ll', 'want', 'know', 'hit', 'left', 'again', 'telling', 'makes', 'realizes',
    'trying', 'part', 'talking',  'took', 'Jamie', 'Waldo', 'coming', 'suggests', 'himself',
    'later', 'having', 'out', 'run', 'its', 'Jessica', 'what\'s',
    'Danny', 'Karl',  'asking', 'tried', 'day', 'sits', 'say', 'says', 'asks',
    'She\'s', 'Sandy', 'Mona', 'Mac', 'show', 'glass', 'until', 'only', 'live', 'called', 'few', 'used', 'Liam',
    'Ffion',  'around', 'meets', 'already', 'eventually', 'explains', 'takes', 'help', 'behind', 'allows', 'plays', 'opens', 'allowing',
    'Nida', 'Gaap', 'Vicky', 'Len', 'Michael', 'Keith', 'Suzie', 'Chris', 'three', 'Joan', 'Salma', 'Krish', 'Eric',
    'She\'s', 'Sandy', 'Mona', 'Mac', 'show', 'watch', 'show.', 'Jack', 'Rachel', 'Catherine', 'Too', 'Munk', 'Kevin', 'music', 'new', 'still', 'play', 'Mia',
    'Shazia', 'Rob', 'picture', 'puts', 'promises', 'woman', 'case', 'grabs', 'phone', 'decides', 'try', 'reveals', 'new', 'Victoria', 'Baxter',
    'nearby', 'calls', 'Bear', 'Jamie', 'public', 'own', 'world', 'Liam', 'back.', 'Kelly', 'time', 'leaves.', 'party', 'real', 'her,'
    'agrees', 'working',  'again.', 'Parke', 'becomes', 'Jem', 'another', 'house',' onto', 'Jamie', 'Ffion', 'Duke', 'Cole','Bing','Katie','herself',
    'together.', 'Duke', 'Cole', 'Katie', 'really', 'said', 'other', 'leave', 'months', 'monkey',
    'Carrie', 'Emily', 'feel', 'Dawson', 'day', 'follows', 'that', 'hasn\'t', 'Daly', 'Baxter',
    'Chris', 'Waldo', 'Jem', 'Liam', 'Rachel', 'Martha', 'Amy', 'Too', 'Kappa', 'Nish', 'Bella',
    'Rolo', 'Pia', 'Gaap', 'Keith', 'Karl', 'Vicky', 'Catherine', 'Coach', 'Salma', 'Kevin',
    'Victoria', 'Yorkie', 'San', 'work', 'killed', 'day', 'that', 'herself', 'suddenly', 'uses'
])

# Load the JSON file
with open('127.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract documents and labels
docs = [item['content'] for item in data]
y_labels = [item['labels'][0] for item in data]  # Assuming each doc has at least one label and taking the first label

# Create a mapping from labels to integers
label_mapping = {label: i for i, label in enumerate(set(y_labels))}

# Convert labels to integers
y = [label_mapping[label] for label in y_labels]

# Prepare our empty sub-models and reduce frequent words while we are at it.
empty_embedding_model = BaseEmbedder()
empty_dimensionality_model = BaseDimensionalityReduction()
empty_cluster_model = BaseCluster()
# Use CountVectorizer to handle stop words
vectorizer_model = CountVectorizer(stop_words=list(stop_words))  # Convert stop_words set to a list
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

# Fit BERTopic without actually performing any clustering
topic_model = BERTopic(
    embedding_model=empty_embedding_model,
    umap_model=empty_dimensionality_model,
    hdbscan_model=empty_cluster_model,
    ctfidf_model=ctfidf_model,  # Pass the ctfidf model
    vectorizer_model=vectorizer_model  # Pass the CountVectorizer instance to BERTopic
)
topics, probs = topic_model.fit_transform(docs, y=y)  # Pass the integer labels to fit_transform

# Print the topics and their probabilities
for topic in topics:
    print(f"Topic: {topic}")

# Print top words for each topic
topic_info = topic_model.get_topic_info()
for index, row in topic_info.iterrows():
    if row['Topic'] != -1:  # Ignore outliers
        print(f"Topic {row['Topic']}: {row['Name']}")


Topic: 0
Topic: 2
Topic: 20
Topic: 19
Topic: 18
Topic: 17
Topic: 16
Topic: 15
Topic: 1
Topic: 2
Topic: 1
Topic: 1
Topic: 14
Topic: 0
Topic: 13
Topic: 12
Topic: 3
Topic: 11
Topic: 0
Topic: 10
Topic: 9
Topic: 8
Topic: 7
Topic: 6
Topic: 5
Topic: 4
Topic: 21
Topic 0: 0_rolo_carrie_nish_joe
Topic 1: 1_kelly_yorkie_stripe_junipero
Topic 2: 2_callow_susannah_demand_ransom
Topic 3: 3_amy_frank_coach_lenny
Topic 4: 4_bo_mazey_hector_whitty
Topic 5: 5_david_cliff_lana_henry
Topic 6: 6_pia_davis_janet_stuart
Topic 7: 7_joan_salma_krish_mac
Topic 8: 8_ashley_rachel_catherine_too
Topic 9: 9_chris_jaden_billy_smithereen
Topic 10: 10_danny_karl_theo_pair
Topic 11: 11_bella_dog_anthony_clarke
Topic 12: 12_mia_shazia_rob_noni
Topic 13: 13_sara_marie_tablet_trick
Topic 14: 14_parke_blue_garrett_adis
Topic 15: 15_lacie_naomi_alan_plummet
Topic 16: 16_jamie_waldo_gwendolyn_napier
Topic 17: 17_victoria_jem_baxter_damien
Topic 18: 18_martha_ash_android_service
Topic 19: 19_ffion_liam_jonas_grain
Topic 20: 2

In [22]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,3,0_rolo_carrie_nish_joe,"[rolo, carrie, nish, joe, clayton, dawson, mat...",[Captain Robert Daly (Jesse Plemons) and his c...
1,1,3,1_kelly_yorkie_stripe_junipero,"[kelly, yorkie, stripe, junipero, san, cooper,...","[Cooper (Wyatt Russell), a man living with his..."
2,2,2,2_callow_susannah_demand_ransom,"[callow, susannah, demand, ransom, broadcast, ...",[Kenny gets a call from his mother. His face i...
3,3,1,3_amy_frank_coach_lenny,"[amy, frank, coach, lenny, match, nicola, syst...",[The coach directs Frank to the hub. He walks ...
4,4,1,4_bo_mazey_hector_whitty,"[bo, mazey, hector, whitty, duke, pictures, te...",[Bo sits in a motel parking lot listening to a...
5,5,1,5_david_cliff_lana_henry,"[david, cliff, lana, henry, station, jessica, ...",[As David Ross is drawing a portrait of his tw...
6,6,1,6_pia_davis_janet_stuart,"[pia, davis, janet, stuart, kenneth, richard, ...",[Pia Koresh sleeps on Davis McCardle's shoulde...
7,7,1,7_joan_salma_krish_mac,"[joan, salma, krish, mac, sandy, tv, eric, hay...",[Joan Tait wakes up to her phone alarm. She tu...
8,8,1,8_ashley_rachel_catherine_too,"[ashley, rachel, catherine, too, jack, munk, k...","[In a busy school cafeteria, Rachel Goggins si..."
9,9,1,9_chris_jaden_billy_smithereen,"[chris, jaden, billy, smithereen, penelope, me...","[In 2018, Chris (Andrew Scott) is a rideshare ..."


In [23]:
# Map input `y` to topics
mappings = topic_model.topic_mapper_.get_mappings()

# Assuming 'data' should have a dictionary called 'target_names'
# If 'target_names' is actually a list, you'll need to adjust how you map the values
if "target_names" in data and isinstance(data["target_names"], dict):
    mappings = {value: data["target_names"][key] for key, value in mappings.items()}
else:
    print("Warning: 'data' does not contain a 'target_names' dictionary.")
    mappings = {value: key for key, value in mappings.items()}  # Use the original label if no mapping is available

# Assign original classes to our topics
df = topic_model.get_topic_info()
df["Class"] = df.Topic.map(mappings)
df

,Topic,Count,Name,Representation,Representative_Docs,Class
0,0,3,0_rolo_carrie_nish_joe,"[rolo, carrie, nish, joe, clayton, dawson, mat...",[Captain Robert Daly (Jesse Plemons) and his c...,19
1,1,3,1_kelly_yorkie_stripe_junipero,"[kelly, yorkie, stripe, junipero, san, cooper,...","[Cooper (Wyatt Russell), a man living with his...",5
2,2,2,2_callow_susannah_demand_ransom,"[callow, susannah, demand, ransom, broadcast, ...",[Kenny gets a call from his mother. His face i...,13
3,3,1,3_amy_frank_coach_lenny,"[amy, frank, coach, lenny, match, nicola, syst...",[The coach directs Frank to the hub. He walks ...,8
4,4,1,4_bo_mazey_hector_whitty,"[bo, mazey, hector, whitty, duke, pictures, te...",[Bo sits in a motel parking lot listening to a...,6
5,5,1,5_david_cliff_lana_henry,"[david, cliff, lana, henry, station, jessica, ...",[As David Ross is drawing a portrait of his tw...,9
6,6,1,6_pia_davis_janet_stuart,"[pia, davis, janet, stuart, kenneth, richard, ...",[Pia Koresh sleeps on Davis McCardle's shoulde...,21
7,7,1,7_joan_salma_krish_mac,"[joan, salma, krish, mac, sandy, tv, eric, hay...",[Joan Tait wakes up to her phone alarm. She tu...,0
8,8,1,8_ashley_rachel_catherine_too,"[ashley, rachel, catherine, too, jack, munk, k...","[In a busy school cafeteria, Rachel Goggins si...",7
9,9,1,9_chris_jaden_billy_smithereen,"[chris, jaden, billy, smithereen, penelope, me...","[In 2018, Chris (Andrew Scott) is a rideshare ...",14
